In [1]:
import pandas as pd 
import datetime as dt
import numpy as np
import keras
import plotly.graph_objects as go

from keras.models import Sequential
import tensorflow as tf
from alphaRNN import *
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.regularizers import l1
from sklearn.metrics import mean_squared_error, r2_score

Using TensorFlow backend.


In [2]:
data = pd.read_csv("hourlySummaries.csv")
data = data.set_index("hour")
data = data[data.columns[1:]]
# data = data.iloc[:1000,]

train_weight = 0.80
split = int(len(data)*train_weight)
df_train = data.iloc[:split].copy()
df_test = data.iloc[split:].copy()

In [3]:
X_train = df_train.iloc[:,list(range(1,32))]
X_test = df_test.iloc[:,list(range(1,32))]

# X_train = X_train.drop("ethPriceUSD", axis=1)
# X_test = X_test.drop("ethPriceUSD", axis = 1)

y_train = df_train["aaveHourlyChange"]
y_test = df_test["aaveHourlyChange"]
# y_train = df_train["aaveDirectionOfChange"]
# y_test = df_test["aaveDirectionOfChange"]

In [4]:
x_train_reg = np.array(X_train)
x_test_reg = np.array(X_test)
y_train_reg = np.array(y_train)
y_test_reg = np.array(y_test)

In [5]:
x_train_reg = np.reshape(x_train_reg, (x_train_reg.shape[0], x_train_reg.shape[1], 1))
x_test_reg = np.reshape(x_test_reg, (x_test_reg.shape[0], x_test_reg.shape[1], 1))

In [10]:
import time
start = time.time()
model = keras.Sequential()
model.add(keras.layers.LSTM(50, activation='relu', input_shape=(x_train_reg.shape[1], 1)))
model.add(keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(x_train_reg, y_train_reg, epochs=1, validation_split=0.2, verbose=1)
run_time = time.time() - start
y_pred = model.predict(x_test_reg)


Train on 6944 samples, validate on 1736 samples
Epoch 1/1
1024/6944 [===>..........................] - ETA: 32s - loss: 123848375304.0000

In [ ]:
print(run_time)

In [7]:
y_pred_train = model.predict(x_train_reg, verbose = 1)
MSE_train= mean_squared_error(y_train,y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("Train MSE: ", MSE_train)
print("Train r2: ", r2_train)

y_pred = model.predict(x_test_reg, verbose = 1)
MSE_test= mean_squared_error(y_test,y_pred)
r2_test = r2_score(y_test, y_pred)

print("Test MSE: ", MSE_test)
print("Test r2: ", r2_test)

1 - (1-r2_test)*(len(y_test_reg)-1)/(len(y_test_reg)-x_test_reg.shape[1]-1)

8680/8680 [==============================] - 9s 1ms/step
Train MSE:  108068567345.63895
Train r2:  -4057284915.019463
2171/2171 [==============================] - 2s 968us/step
Test MSE:  52913209290.45606
Test r2:  -10969166281.267998


-11128139705.648693

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                10400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [9]:
y_pred_train = model.predict(x_train_reg, verbose = 1)
MSE_train= mean_squared_error(y_train,y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("Train MSE: ", MSE_train)
print("Train r2: ", r2_train)

y_pred = model.predict(x_test_reg, verbose = 1)
MSE_test= mean_squared_error(y_test,y_pred)
r2_test = r2_score(y_test, y_pred)

print("Test MSE: ", MSE_test)
print("Test r2: ", r2_test)

8680/8680 [==============================] - 9s 987us/step
Train MSE:  108068567345.63895
Train r2:  -4057284915.019463
2171/2171 [==============================] - 2s 1ms/step
Test MSE:  52913209290.45606
Test r2:  -10969166281.267998
